In [1]:
# SQL Alchemy

# psycopg2

In [22]:
import psycopg2 as pg

import pandas as pd
import numpy as np

In [3]:
# Credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username=''
pwd = ''

In [26]:
conn = pg.connect(user=username,
          password = pwd,
          host=host,
          port=port,
          database=database)

# 1.0. Schema Query

In [5]:
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute(query_schema)

record = cursor.fetchall()
print(record)

cursor.close()
conn.close()

# 2.0. Table Query

In [16]:
cursor = conn.cursor()

query_table = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname = 'pa004'
"""

cursor.execute(query_table)

record = cursor.fetchall()
print(record)

cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


# 3.0. Collect Data (by using Psycopg2)

In [21]:
cursor = conn.cursor()

query_table = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

cursor.execute(query_table)

record = cursor.fetchall()
print(record)

cursor.close()
conn.close()

[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [23]:
data = pd.DataFrame(record)
data.head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


# 4.0. Collect Data (by using Pandas)

In [24]:
import pandas.io.sql as psql

In [31]:
query_table_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON (u.id = v.id)
                       LEFT JOIN pa004.insurance i ON (u.id = i.id)
    WHERE u.age > 44
    LIMIT 15
"""

df = psql.read_sql(query_table_users, conn)
df.head()

/home/leassis/.pyenv/versions/3.10.5/envs/healthins/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,26021,Male,53,43.0,124.0,26021,1,1-2 Year,Yes,26021,0,36635.0,202,0
1,26042,Male,47,29.0,156.0,26042,1,1-2 Year,Yes,26042,0,2630.0,15,0
2,26061,Male,45,28.0,52.0,26061,1,1-2 Year,Yes,26061,0,99999.0,137,0
3,26101,Female,45,28.0,125.0,26101,1,1-2 Year,Yes,26101,0,43325.0,251,0
4,26146,Male,46,40.0,124.0,26146,1,1-2 Year,Yes,26146,0,2630.0,273,0


In [33]:
df1 = pd.read_sql(query_table_users, conn)
df1 = df1.drop(df.columns[[5, 9]], axis=1)
df1.head()

/home/leassis/.pyenv/versions/3.10.5/envs/healthins/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,51,28.0,124.0,1,1-2 Year,Yes,0,38341.0,49,1
1,Male,62,35.0,10.0,1,> 2 Years,Yes,0,41895.0,67,0
2,Male,59,28.0,26.0,1,1-2 Year,Yes,1,36986.0,168,0
3,Male,63,28.0,124.0,1,1-2 Year,Yes,0,53828.0,103,0
4,Male,66,36.0,26.0,1,1-2 Year,No,1,40181.0,75,0
